In [10]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import re

In [11]:
XF = pd.read_csv("00_Stacked_Dataframe.csv")

#### Only Considering Stations within Washington DC Boundary

In [12]:
Station_shp = pd.read_csv("./XX_Processed_Data/A0_WashingtonDC_StationIDs_01.csv")

In [13]:
XF.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'Hour_(Starting)', 'Year', 'StationID_YearHour'],
      dtype='object')

In [14]:
Station_shp.columns

Index(['start_station', 'Station_ID', 'start_lat', 'start_lng', 'Neighb_Cls',
       'Ward'],
      dtype='object')

### Adding Season Column

In [15]:
# Convert the column to datetime
XF['started_at'] = pd.to_datetime(XF['started_at'], format='%Y-%m-%d %H:%M:%S')
XF['ended_at'] = pd.to_datetime(XF['ended_at'], format='%Y-%m-%d %H:%M:%S')

In [16]:
filt = (XF["started_at"].dt.month >= 3) & (XF['started_at'].dt.month <= 5)
XF.loc[filt,"Season"] = "Spring"

In [17]:
filt = (XF["started_at"].dt.month >= 6) & (XF['started_at'].dt.month <= 8)
XF.loc[filt,"Season"] = "Summer"

In [18]:
filt = (XF["started_at"].dt.month >= 9) & (XF['started_at'].dt.month <= 11)
XF.loc[filt,"Season"] = "Fall"

In [19]:
filt1 = (XF["started_at"].dt.month >= 2)
filt2 = (XF['started_at'].dt.month == 12)
filt = filt1 & filt2

XF.loc[filt,"Season"] = "Winter"

In [20]:
XF["Season"] = XF["Season"].fillna("Winter")

In [21]:
XF["Station_ID"] = XF["start_station_id"]

In [22]:
# Define a function to label each date as weekday or weekend
def categorize_weekday_or_weekend(date):
    if date.weekday() < 5:  # 0-4 represent Monday to Friday (weekdays)
        return 'Weekday'
    else:
        return 'Weekend'

# Apply the function to create a new column 'Day_Type'
XF['Day_Type'] = XF['started_at'].apply(categorize_weekday_or_weekend)

In [23]:
# Merge the DataFrames based on the common fields 'ID'
merged_df = pd.merge(XF, Station_shp, on='Station_ID')

In [24]:
merged_df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat_x', 'start_lng_x', 'end_lat', 'end_lng',
       'member_casual', 'Hour_(Starting)', 'Year', 'StationID_YearHour',
       'Season', 'Station_ID', 'Day_Type', 'start_station', 'start_lat_y',
       'start_lng_y', 'Neighb_Cls', 'Ward'],
      dtype='object')

In [25]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6128884 entries, 0 to 6128883
Data columns (total 24 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    int64         
 6   end_station_name    object        
 7   end_station_id      int64         
 8   start_lat_x         float64       
 9   start_lng_x         float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  Hour_(Starting)     int64         
 14  Year                int64         
 15  StationID_YearHour  object        
 16  Season              object        
 17  Station_ID          int64         
 18  Day_Type            object        
 19  start_station       object        
 20  st

In [229]:
merged_df['end_station_id'] = (merged_df['end_station_id'].astype(int)).astype(str)

In [55]:
merged_df['End_StationID_YearHour_Month'] = merged_df['end_station_id'] + "_" + merged_df['Year'].astype(str) + "_" + merged_df['Hour_(Starting)'].astype(str) + "_" + merged_df["started_at"].dt.month.astype(str) + "_" + merged_df["Day_Type"]
merged_df["Start_StationID_YearHour_Month"] = merged_df["StationID_YearHour"] + "_" + merged_df["started_at"].dt.month.astype(str) + "_" + merged_df["Day_Type"]

In [56]:
merged_df['Start_StationID_YearHour_Month']

0          31660_2021_21_7_Weekend
1          31660_2021_16_7_Weekend
2          31660_2021_21_7_Weekend
3          31660_2021_19_7_Weekday
4          31660_2021_19_7_Weekend
                    ...           
6128879    31426_2023_21_7_Weekday
6128880    31426_2023_21_7_Weekday
6128881    31426_2023_11_7_Weekday
6128882     31426_2023_8_7_Weekday
6128883    31426_2023_11_7_Weekday
Name: Start_StationID_YearHour_Month, Length: 6128884, dtype: object

## 1.0 Master csv File Development

In [190]:
filt1 = merged_df["started_at"].dt.hour == 23
# filt2 = merged_df["Year"] == 2021

x = merged_df[ filt1 ]
Start_Stats = x["Start_StationID_YearHour_Month"].value_counts()
End_Stats = x['End_StationID_YearHour_Month'].value_counts()

Start_DF = pd.DataFrame(Start_Stats)
End_DF = pd.DataFrame(End_Stats)

In [191]:
End_DF

End_DF = End_DF.reset_index()
# y["Count"] =  y["StationID_YearHour"]


End_DF.rename(columns = {"End_StationID_YearHour_Month": "Arrival Count"}, inplace=True)
# y.drop(columns = {"StationID_YearHour"}, inplace=True)
End_DF

,index,Arrival Count
0,31627_2023_23_7_Weekday,72
1,31241_2022_23_9_Weekday,70
2,31330_2022_23_9_Weekday,68
3,31266_2021_23_8_Weekday,68
4,31247_2021_23_7_Weekday,68
...,...,...
14299,31637_2022_23_11_Weekend,1
14300,31826_2023_23_7_Weekday,1
14301,31806_2022_23_12_Weekday,1
14302,31801_2023_23_5_Weekend,1


In [192]:
Start_DF = Start_DF.reset_index()
# y["Count"] =  y["StationID_YearHour"]


Start_DF.rename(columns = {"Start_StationID_YearHour_Month": "Deperture Count"}, inplace=True)
# y.drop(columns = {"StationID_YearHour"}, inplace=True)
Start_DF

,index,Deperture Count
0,31609_2023_23_5_Weekday,138
1,31114_2023_23_6_Weekday,104
2,31209_2022_23_8_Weekday,99
3,31203_2023_23_5_Weekday,95
4,31609_2022_23_6_Weekday,91
...,...,...
12632,31417_2023_23_5_Weekend,1
12633,31417_2023_23_3_Weekday,1
12634,31417_2023_23_3_Weekend,1
12635,31417_2023_23_2_Weekday,1


In [193]:
# Merge the DataFrames based on the common fields 'ID'
Start_End_XX = pd.merge(Start_DF, End_DF, on='index')

In [194]:
# Concatenate df2 under df1
Start_End_DF = pd.concat([Start_End_DF, Start_End_XX], ignore_index=True)

In [195]:
Start_End_DF

,index,Deperture Count,Arrival Count
0,31114_2022_0_8_Weekend,86,26
1,31114_2023_0_5_Weekend,81,25
2,31114_2021_0_7_Weekend,81,34
3,31114_2021_0_10_Weekend,75,36
4,31114_2023_0_4_Weekend,74,30
...,...,...,...
279251,31417_2023_23_5_Weekend,1,2
279252,31417_2023_23_3_Weekday,1,7
279253,31417_2023_23_3_Weekend,1,3
279254,31417_2023_23_2_Weekday,1,3


In [196]:
Start_End_DF[['StationID', 'Year', 'Hour', 'Month', "Day_Type"]] = Start_End_DF['index'].str.split('_', expand=True)
Start_End_DF.drop(columns = ["index"] )

,Deperture Count,Arrival Count,StationID,Year,Hour,Month,Day_Type
0,86,26,31114,2022,0,8,Weekend
1,81,25,31114,2023,0,5,Weekend
2,81,34,31114,2021,0,7,Weekend
3,75,36,31114,2021,0,10,Weekend
4,74,30,31114,2023,0,4,Weekend
...,...,...,...,...,...,...,...
279251,1,2,31417,2023,23,5,Weekend
279252,1,7,31417,2023,23,3,Weekday
279253,1,3,31417,2023,23,3,Weekend
279254,1,3,31417,2023,23,2,Weekday


In [206]:
len(Start_End_DF["StationID"].unique())

359

In [208]:
Start_End_DF.drop(columns = ["index"], inplace = True)

In [209]:
Station_shp["StationID"] = Station_shp["Station_ID"].astype(str)

In [210]:
# Merge the DataFrames based on the common fields 'ID'
Merged_Start_End_DF = pd.merge(Start_End_DF, Station_shp, on='StationID')

In [211]:
Merged_Start_End_DF.head()

,Deperture Count,Arrival Count,StationID,Year,Hour,Month,Day_Type,start_station,Station_ID,start_lat,start_lng,Neighb_Cls,Ward
0,86,26,31114,2022,0,8,Weekend,18th St & Wyoming Ave NW,31114,38.918809,-77.041571,Cluster 1,Ward 1
1,81,25,31114,2023,0,5,Weekend,18th St & Wyoming Ave NW,31114,38.918809,-77.041571,Cluster 1,Ward 1
2,81,34,31114,2021,0,7,Weekend,18th St & Wyoming Ave NW,31114,38.918809,-77.041571,Cluster 1,Ward 1
3,75,36,31114,2021,0,10,Weekend,18th St & Wyoming Ave NW,31114,38.918809,-77.041571,Cluster 1,Ward 1
4,74,30,31114,2023,0,4,Weekend,18th St & Wyoming Ave NW,31114,38.918809,-77.041571,Cluster 1,Ward 1


In [354]:
Merged_Start_End_DF.to_csv("61Merged_Start_End_DF.csv")

## 2.0 Associate csv File on Seasons

In [3]:
XF = pd.read_csv("00_Stacked_Dataframe.csv")

In [4]:
Station_shp = pd.read_csv("./XX_Processed_Data/A0_WashingtonDC_StationIDs_01.csv")

In [5]:
# Convert the column to datetime
XF['started_at'] = pd.to_datetime(XF['started_at'], format='%Y-%m-%d %H:%M:%S')
XF['ended_at'] = pd.to_datetime(XF['ended_at'], format='%Y-%m-%d %H:%M:%S')

XF["Station_ID"] = XF["start_station_id"]

In [6]:
# Merge the DataFrames based on the common fields 'ID'
merged_df = pd.merge(XF, Station_shp, on='Station_ID')

In [9]:
merged_df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat_x', 'start_lng_x', 'end_lat', 'end_lng',
       'member_casual', 'Hour_(Starting)', 'Year', 'StationID_YearHour',
       'Station_ID', 'start_station', 'start_lat_y', 'start_lng_y',
       'Neighb_Cls', 'Ward', 'Day_Type'],
      dtype='object')

In [7]:
# Define a function to label each date as weekday or weekend
def categorize_weekday_or_weekend(date):
    if date.weekday() < 5:  # 0-4 represent Monday to Friday (weekdays)
        return 'Weekday'
    else:
        return 'Weekend'

# Apply the function to create a new column 'Day_Type'
merged_df['Day_Type'] = merged_df['started_at'].apply(categorize_weekday_or_weekend)

In [26]:
merged_df['end_station_id'] = (merged_df['end_station_id'].astype(int)).astype(str)
merged_df['End_StationID_YearHour_Season'] = merged_df['end_station_id'] + "_" + merged_df['Year'].astype(str) + "_" + merged_df['Hour_(Starting)'].astype(str) + "_" + merged_df["Season"].astype(str) + "_" + merged_df["Day_Type"] 
merged_df["Start_StationID_YearHour_Season"] = merged_df["StationID_YearHour"] + "_" + merged_df["Season"].astype(str) + "_" + merged_df["Day_Type"]

In [28]:
merged_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat_x,start_lng_x,...,Season,Station_ID,Day_Type,start_station,start_lat_y,start_lng_y,Neighb_Cls,Ward,End_StationID_YearHour_Season,Start_StationID_YearHour_Season
0,FF02EE3AE115F951,classic_bike,2021-07-04 21:57:56,2021-07-04 22:00:43,8th & K St NE,31660,6th & K St NE,31645,38.902732,-76.99515,...,Summer,31660,Weekend,8th & K St NE,38.902657,-76.995111,Cluster 25,Ward 6,31645_2021_21_Summer_Weekend,31660_2021_21_Summer_Weekend
1,EAD3303AA5518C14,classic_bike,2021-07-24 16:26:54,2021-07-24 16:44:57,8th & K St NE,31660,10th & U St NW,31111,38.902732,-76.99515,...,Summer,31660,Weekend,8th & K St NE,38.902657,-76.995111,Cluster 25,Ward 6,31111_2021_16_Summer_Weekend,31660_2021_16_Summer_Weekend
2,4CAAF169950650A7,classic_bike,2021-07-17 21:00:02,2021-07-17 21:21:27,8th & K St NE,31660,10th & U St NW,31111,38.902732,-76.99515,...,Summer,31660,Weekend,8th & K St NE,38.902657,-76.995111,Cluster 25,Ward 6,31111_2021_21_Summer_Weekend,31660_2021_21_Summer_Weekend
3,2474026AF600F1FF,classic_bike,2021-07-09 19:10:59,2021-07-09 19:28:11,8th & K St NE,31660,1st & D St SE,31643,38.902732,-76.99515,...,Summer,31660,Weekday,8th & K St NE,38.902657,-76.995111,Cluster 25,Ward 6,31643_2021_19_Summer_Weekday,31660_2021_19_Summer_Weekday
4,E5C209B34B279DB7,classic_bike,2021-07-25 19:05:05,2021-07-25 19:24:10,8th & K St NE,31660,1st & K St SE,31628,38.902732,-76.99515,...,Summer,31660,Weekend,8th & K St NE,38.902657,-76.995111,Cluster 25,Ward 6,31628_2021_19_Summer_Weekend,31660_2021_19_Summer_Weekend


In [29]:
Start_End_DF = pd.DataFrame()

In [30]:
Hours = list(range(0,24))

for i in Hours:
    filt1 = merged_df["started_at"].dt.hour == i

    x = merged_df[ filt1 ]
    Start_Stats = x["Start_StationID_YearHour_Season"].value_counts()
    End_Stats = x['End_StationID_YearHour_Season'].value_counts()

    Start_DF = pd.DataFrame(Start_Stats)
    End_DF = pd.DataFrame(End_Stats)
    
    End_DF = End_DF.reset_index()
    End_DF.rename(columns = {"End_StationID_YearHour_Season": "Arrival Count"}, inplace=True)
    
    Start_DF = Start_DF.reset_index()
    Start_DF.rename(columns = {"Start_StationID_YearHour_Season": "Deperture Count"}, inplace=True)
    
    # Merge the DataFrames based on the common fields 'ID'
    Start_End_XX = pd.merge(Start_DF, End_DF, on='index')

    # Concatenate df2 under df1
    Start_End_DF = pd.concat([Start_End_DF, Start_End_XX], ignore_index=True)

In [31]:
Start_End_DF

,index,Deperture Count,Arrival Count
0,31114_2022_0_Summer_Weekend,208,80
1,31114_2023_0_Spring_Weekend,186,61
2,31202_2022_0_Summer_Weekend,168,79
3,31114_2022_0_Fall_Weekend,152,61
4,31114_2021_0_Fall_Weekend,144,76
...,...,...,...
128546,31665_2021_23_Summer_Weekday,1,8
128547,31530_2023_23_Winter_Weekend,1,4
128548,31530_2022_23_Fall_Weekend,1,2
128549,31530_2022_23_Spring_Weekend,1,5


In [32]:
Start_End_DF[['StationID', 'Year', 'Hour', 'Season', 'Day_Type']] = Start_End_DF['index'].str.split('_', expand=True)
Start_End_DF.drop(columns = ["index"], inplace=True )

In [33]:
Start_End_DF

,Deperture Count,Arrival Count,StationID,Year,Hour,Season,Day_Type
0,208,80,31114,2022,0,Summer,Weekend
1,186,61,31114,2023,0,Spring,Weekend
2,168,79,31202,2022,0,Summer,Weekend
3,152,61,31114,2022,0,Fall,Weekend
4,144,76,31114,2021,0,Fall,Weekend
...,...,...,...,...,...,...,...
128546,1,8,31665,2021,23,Summer,Weekday
128547,1,4,31530,2023,23,Winter,Weekend
128548,1,2,31530,2022,23,Fall,Weekend
128549,1,5,31530,2022,23,Spring,Weekend


In [62]:
# Start_End_DF.to_csv("64For_Seasonal_Hourly_DayType_Dynamics_Year.csv")

## Taking Year wise Average Values

In [43]:
Stations = list(Start_End_DF["StationID"].unique())
Seasons = list(Start_End_DF["Season"].unique())
Hours = list(Start_End_DF["Hour"].unique())
Day_Type = list(Start_End_DF["Day_Type"].unique())

In [64]:
Day_Type[1]

'Weekday'

In [45]:
X = Start_End_DF.groupby(['StationID']).get_group("31114")

In [50]:
X = Start_End_DF.groupby(['StationID']).get_group("31114")
filt1 = X["Season"] == Seasons[0]
filt2 = X["Hour"] == Hours[17]
# filt3 = X["Day_Type"] == Day_Type[0]
# df = X.loc[filt1 & filt2 & filt3]
df = X.loc[filt1 & filt2]

DCount = df["Deperture Count"].agg("mean")
ACount = df["Arrival Count"].agg("mean")
df

,Deperture Count,Arrival Count,StationID,Year,Hour,Season,Day_Type
87273,272,384,31114,2022,17,Summer,Weekday
87559,210,265,31114,2023,17,Summer,Weekday
88056,148,152,31114,2021,17,Summer,Weekday
88529,107,115,31114,2022,17,Summer,Weekend
89483,60,77,31114,2023,17,Summer,Weekend
89857,49,62,31114,2021,17,Summer,Weekend


In [56]:
Seasonal_Hour_DF = pd.DataFrame()

In [65]:
# for i in Stations:
#     X = Start_End_DF.groupby(['StationID']).get_group(i)
    
#     for j in Seasons:
#         filt1 = X["Season"] == j
        
#         for m in Day_Type:
#             filt3 = X["Day_Type"] == m
        
#         for k in Hours:
#             filt2 = X["Hour"] == k
#             df = X.loc[filt1 & filt2 & filt3]
            
#             df.size
#             if df.size == 0:
#                 print(f"{k} hours of {i} station in {m} on {j} does not have any entry")
#                 break

#             DCount = df["Deperture Count"].agg("mean")
#             ACount = df["Arrival Count"].agg("mean")

#             #Hour
#             Hour = k
#             #Station_ID
#             StationID = i
#             #Season
#             Season = j
#             #DayType
#             DayType = m
            
#             # Create a new row as a dictionary
#             new_row = {"Hour_(Starting)" : Hour,
#                        'StationID' : StationID,
#                        "Season" : Season,
#                        "Day_Type" : DayType,
#                        "Deperture Count" : DCount, 
#                        "Arrival Count" : ACount}

#             x = (pd.DataFrame([new_row]))
#             Seasonal_Hour_DF = pd.concat([Seasonal_Hour_DF,x], axis=0)

In [69]:
for i in Stations:
    X = Start_End_DF.groupby(['StationID']).get_group(i)
    
    for j in Seasons:
        filt1 = X["Season"] == j
        filt3 = X["Day_Type"] == Day_Type[0]
        
        for k in Hours:
            filt2 = X["Hour"] == k
            df = X.loc[filt1 & filt2 & filt3]
            
            df.size
            if df.size == 0:
                print(f"{k} hours of {i} station in {Day_Type[0]} on {j} does not have any entry")
                break

            DCount = df["Deperture Count"].agg("mean")
            ACount = df["Arrival Count"].agg("mean")

            #Hour
            Hour = k
            #Station_ID
            StationID = i
            #Season
            Season = j
            #DayType
            DayType = Day_Type[0]
            
            # Create a new row as a dictionary
            new_row = {"Hour_(Starting)" : Hour,
                       'StationID' : StationID,
                       "Season" : Season,
                       "Day_Type" : DayType,
                       "Deperture Count" : DCount, 
                       "Arrival Count" : ACount}

            x = (pd.DataFrame([new_row]))
            Seasonal_Hour_DF = pd.concat([Seasonal_Hour_DF,x], axis=0)

4 hours of 31114 station in Weekday on Winter does not have any entry
5 hours of 31202 station in Weekday on Spring does not have any entry
5 hours of 31327 station in Weekday on Fall does not have any entry
5 hours of 31327 station in Weekday on Winter does not have any entry
5 hours of 31109 station in Weekday on Spring does not have any entry
5 hours of 31109 station in Weekday on Winter does not have any entry
5 hours of 31102 station in Weekday on Summer does not have any entry
4 hours of 31102 station in Weekday on Winter does not have any entry
4 hours of 31104 station in Weekday on Spring does not have any entry
5 hours of 31104 station in Weekday on Fall does not have any entry
4 hours of 31104 station in Weekday on Winter does not have any entry
4 hours of 31266 station in Weekday on Winter does not have any entry
4 hours of 31289 station in Weekday on Fall does not have any entry
3 hours of 31289 station in Weekday on Winter does not have any entry
5 hours of 31201 station i

In [70]:
len(Seasonal_Hour_DF.StationID.unique())

352

In [73]:
Seasonal_Hour_DF["Deperture Count"] = Seasonal_Hour_DF["Deperture Count"].astype(int)
Seasonal_Hour_DF["Arrival Count"] = Seasonal_Hour_DF["Arrival Count"].astype(int)

In [74]:
Seasonal_Hour_DF

,Hour_(Starting),StationID,Season,Day_Type,Deperture Count,Arrival Count
0,0,31114,Summer,Weekday,67,33
0,1,31114,Summer,Weekday,44,12
0,2,31114,Summer,Weekday,12,4
0,3,31114,Summer,Weekday,4,2
0,4,31114,Summer,Weekday,3,1
...,...,...,...,...,...,...
0,0,31716,Winter,Weekend,1,2
0,0,31521,Spring,Weekend,1,1
0,0,31521,Fall,Weekend,1,9
0,1,31521,Fall,Weekend,1,2


In [75]:
Seasonal_Hour_DF["Day_Type"].unique()

array(['Weekday', 'Weekend'], dtype=object)

In [456]:
Seasonal_Hour_DF.to_csv("62For_Seasonal_Hourly_Dynamics.csv")

In [77]:
# Seasonal_Hour_DF.to_csv("63_1For_Seasonal_Hourly_DayType_Dynamics.csv")